In [4]:
%load_ext autoreload
%autoreload 2

In [12]:
from src.dataset.sgpt import SGPTDataset, SGPTIterableDataset
from src.task.optimus_v2 import OptimusTask
from src.task.sgpt import SGPTTask

In [ ]:
vae = OptimusTask.load_from_checkpoint("checkpoint/optimus-v2-44M/optimus-v2-tiny-vae.ckpt")

In [15]:
dataset = SGPTDataset(
    vae,
    ["heegyu/aihub_daily_conv_2022"],
    16,
    column="utterances"
)


In [18]:
dataset[0]

{'inputs': tensor([[ 0.4252, -0.5721,  1.1754,  ...,  0.0608, -0.0282,  0.0524],
         [ 0.1301,  0.5770,  0.2209,  ...,  0.2190, -0.0024, -0.5700],
         [ 0.2236,  0.2152,  0.4862,  ...,  0.2209, -0.4296, -0.1005],
         ...,
         [ 0.2243,  0.8908, -0.1080,  ...,  0.6288, -0.2741, -0.1808],
         [-0.1994,  0.6122, -0.0990,  ...,  0.0792, -0.4221, -0.2426],
         [ 0.1913, -0.1042,  0.5557,  ..., -0.2009,  0.1301, -0.3088]]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor([[ 0.1301,  0.5770,  0.2209,  ...,  0.2190, -0.0024, -0.5700],
         [ 0.2236,  0.2152,  0.4862,  ...,  0.2209, -0.4296, -0.1005],
         [-0.2920,  0.2550,  0.2707,  ...,  0.1403, -0.1182, -0.3284],
         ...,
         [-0.1994,  0.6122, -0.0990,  ...,  0.0792, -0.4221, -0.2426],
         [ 0.1913, -0.1042,  0.5557,  ..., -0.2009,  0.1301, -0.3088],
         [-0.0351,  0.3009, -0.4232,  ...,  0.2411, -0.2679, -0.4061]])}

In [33]:
item["inputs"].shape, item["attention_mask"].shape, item["labels"].shape

(torch.Size([16, 512]), torch.Size([16]), torch.Size([16, 512]))

In [1]:
from transformers import GPT2Config, GPT2Model

config = GPT2Config.from_json_file("config/model/sgpt-tiny.json")
gpt = GPT2Model(config)
gpt

GPT2Model(
  (wte): Embedding(1, 512)
  (wpe): Embedding(16, 512)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affi

In [21]:
import torch.nn.functional as F

def step(batch, batch_idx) -> dict:
    mask = batch["attention_mask"]
    hidden = gpt(
        inputs_embeds=batch["inputs"],
        attention_mask=mask
        ).last_hidden_state
    outputs = F.mse_loss(hidden, batch["labels"], reduce=False).mean(-1)
    outputs = outputs.masked_fill(mask == 0, 0)
    loss = outputs.sum() / mask.sum()

    return loss

item = dataset[0]
for k, v in item.items():
    item[k] = v.unsqueeze(0)
step(item, 0)

tensor(0.9486, grad_fn=<DivBackward0>)